In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms,datasets
import os

In [2]:
BATCH_SIZE = 64000
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print (DEVICE)
EPOCHS = 100

cuda


In [3]:
pipline =transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307),(0.3081)),
])

In [4]:

train_set = datasets.MNIST("data",train=True,download=True,transform=pipline)
test_set = datasets.MNIST("data",train=False,download=True,transform=pipline)


In [5]:
from torch.utils.data import DataLoader
train_loader = DataLoader(train_set,batch_size=BATCH_SIZE, shuffle=True, num_workers=56)
test_loader = DataLoader(test_set,batch_size=BATCH_SIZE, shuffle=True, num_workers=56)

In [7]:
class Digit(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,32,3,padding= 1 )
        self.conv2 = nn.Conv2d(32,64,3,padding= 1 )
        self.fc1 = nn.Linear(64*14*14, 500)
        self.fc2 = nn.Linear(500, 10)
    def forward(self, x):
        inpute_size = x.size(0)
        x = self.conv1(x)  
        x = F.relu(x) #非线性
        x = F.max_pool2d(x,2,2)
        x = self.conv2(x)
        x = F.relu(x)
        x = x.view(inpute_size,-1) # -1自动计算
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        output = F.log_softmax(x,dim=1) #计算分类后的概率
        return output


In [8]:
model = Digit().to(DEVICE)
optimizer = optim.Adam(model.parameters())
if torch.cuda.device_count() > 1:
    print("Using", torch.cuda.device_count(), "GPUs")
    model = nn.DataParallel(model)

Using 2 GPUs


In [9]:
def train_modle(model ,device , train_loader , optimizer , epoch):
    model.train()
    for batch_index, (data ,target) in enumerate(train_loader):
        data, target = data.to(device),target.to(device)
        optimizer.zero_grad()
        output = model(data)
        lose = F.cross_entropy(output, target)
       # pred = output.max(1,keepdim= True)
        lose.backward()
        optimizer.step()
        if batch_index % 3000 ==  0 :
            print ("train :{} \t loss : {:.6f}".format(epoch ,lose.item()))



In [1]:
def test_modle(model , device , test_loader):
    model.eval()
    correct = 0.0  
    test_loss =0.0
    with torch.no_grad() :
        for data , target in test_loader :
            data , target =data.to(device), target.to(device)
            output = model(data)
            test_loss +=F.cross_entropy(output , target).item()
            pred = output.max(1,keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
        test_loss /= len(test_loader.dataset)
        print ("loss: {:.4f},rate : {:.3f}\n".format(test_loss,100*correct/len(test_loader.dataset)))

In [20]:
for epoch in range (1,EPOCHS+1):
    train_modle(model ,DEVICE ,train_loader ,optimizer,epoch)
    test_modle(model , DEVICE,test_loader)

train :1 	 loss : 2.304159
loss: 0.0002,rate : 55.180

train :2 	 loss : 1.715967
loss: 0.0001,rate : 60.670

train :3 	 loss : 1.268524
loss: 0.0001,rate : 70.990

train :4 	 loss : 1.031204
loss: 0.0001,rate : 76.150

train :5 	 loss : 0.790404
loss: 0.0001,rate : 81.770

train :6 	 loss : 0.591556
loss: 0.0001,rate : 82.730

train :7 	 loss : 0.569559
loss: 0.0000,rate : 86.770

train :8 	 loss : 0.465474
loss: 0.0000,rate : 89.020

train :9 	 loss : 0.400895
loss: 0.0000,rate : 88.340

train :10 	 loss : 0.414433
loss: 0.0000,rate : 89.070

train :11 	 loss : 0.393104
loss: 0.0000,rate : 90.550

train :12 	 loss : 0.341101
loss: 0.0000,rate : 91.640

train :13 	 loss : 0.308841
loss: 0.0000,rate : 91.970

train :14 	 loss : 0.298908
loss: 0.0000,rate : 92.060

train :15 	 loss : 0.293958
loss: 0.0000,rate : 92.270

train :16 	 loss : 0.281416
loss: 0.0000,rate : 92.980

train :17 	 loss : 0.260936
loss: 0.0000,rate : 93.480

train :18 	 loss : 0.241487
loss: 0.0000,rate : 93.810

t